In [5]:
import requests
from bs4 import BeautifulSoup, NavigableString
import re


# Define the URL for the Wikimedia REST API
api_url = "https://es.wikipedia.org/api/rest_v1/page/random/html/"

# Initialize variables
accumulated_text = ""
request_count = 0

# This receives a paragraph element
def process_paragraph(paragraph):
    sentences = {}
    count = 1

    # loop through the children of the para element
    for content in paragraph.contents:
        # if the current count value is not a key already, create a new key value pair
        if count not in sentences:
            sentences[count] = []

        sentences[count] += content
        if isinstance(content, NavigableString):
            if str(content).endswith(('!', '?', '.')):
                count += 1
    
    print(sentences)

    '''# Add the last sentence if it's not empty
    if sentence:
        sentences.append(sentence)'''

    '''return ' '.join(sentences)'''

# Use a while loop to keep making requests until the accumulated text length is at least 1000 characters
while len(accumulated_text) < 50000:
    # Send a GET request to the API
    response = requests.get(api_url)
    request_count += 1

    # Check if the request was successful
    if response.status_code == 200:
        # Get the HTML content of the page
        html_content = response.content

        # Use BeautifulSoup to parse the HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Process each paragraph
        paragraphs = soup.find_all('p')
        list_of_dicts = []
        for p in paragraphs:
            processed_text = process_paragraph(p)
            list_of_dicts.append(processed_text)
            '''accumulated_text += processed_text + " "'''
        
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        break

# Once the desired length is reached
if len(accumulated_text) >= 50000:
    # Count the occurrences of 'por' as a separate word
    por_count = len(re.findall(r'\bpor\b', accumulated_text, re.IGNORECASE))

    # Save the text to a file
    with open('wikimedia_text.txt', 'w', encoding='utf-8') as file:
        file.write(accumulated_text)

    print(f"Text saved after {request_count} requests.")
    print(f"The word 'por' occurs {por_count} times in the text.")
else:
    print("Unable to retrieve sufficient text.")


> /var/folders/t2/f6pwz4s16_bghsz2cxpp3s1r0000gn/T/ipykernel_74349/3735792465.py(30)process_paragraph()
     28 
     29     pdb.set_trace()
---> 30     print(sentences)
     31 
     32     '''# Add the last sentence if it's not empty



In [46]:
import requests
from bs4 import BeautifulSoup
import re

# Define the URL for the Wikimedia REST API
api_url = "https://es.wikipedia.org/api/rest_v1/page/random/html/"

# Initialize variables
accumulated_text = ""
request_count = 0

# Function to split paragraph into sentences and exclude sentences with links
def process_paragraph(paragraph):
    sentences = re.split(r'(?<=[.!?])\s+', paragraph.get_text(strip=True))
    print(f'PARAGRAPH: {paragraph}')
    print(f'SENTENCES: {sentences}')
    return ' '.join(sentence for sentence in sentences if paragraph.find('a', string=sentence) is None)

# Use a while loop to keep making requests until the accumulated text length is at least 1000 characters
while len(accumulated_text) < 1000:
    # Send a GET request to the API
    response = requests.get(api_url)
    request_count += 1

    # Check if the request was successful
    if response.status_code == 200:
        # Get the HTML content of the page
        html_content = response.content

        # Use BeautifulSoup to parse the HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Process each paragraph
        paragraphs = soup.find_all('p')
        for p in paragraphs:
            processed_text = process_paragraph(p)
            accumulated_text += processed_text + " "

        print(f"Accumulated text length: {len(accumulated_text)} (after {request_count} requests)")
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        break

# Once the desired length is reached
if len(accumulated_text) >= 1000:
    # Count the occurrences of 'por' as a separate word
    por_count = len(re.findall(r'\bpor\b', accumulated_text, re.IGNORECASE))

    # Save the text to a file
    with open('wikimedia_text.txt', 'w', encoding='utf-8') as file:
        file.write(accumulated_text)

    print(f"Text saved after {request_count} requests.")
    print(f"The word 'por' occurs {por_count} times in the text.")
else:
    print("Unable to retrieve sufficient text.")


PARAGRAPH: <p id="mwEg">El martes <a href="./2_de_julio" id="mwEw" rel="mw:WikiLink" title="2 de julio">2 de julio</a> de <a href="./2019" id="mwFA" rel="mw:WikiLink" title="2019">2019</a> se produjo un <b id="mwFQ"><a href="./Eclipse_solar" id="mwFg" rel="mw:WikiLink" title="Eclipse solar">eclipse solar</a> total</b> que fue visto a lo largo del <a href="./Océano_Pacífico" id="mwFw" rel="mw:WikiLink" title="Océano Pacífico">océano Pacífico</a>, en <a href="./Chile" id="mwGA" rel="mw:WikiLink" title="Chile">Chile</a> y, al atardecer, en <a href="./Argentina" id="mwGQ" rel="mw:WikiLink" title="Argentina">Argentina</a> y <a href="./Uruguay" id="mwGg" rel="mw:WikiLink" title="Uruguay">Uruguay</a>.</p>
SENTENCES: ['El martes2 de juliode2019se produjo uneclipse solartotalque fue visto a lo largo delocéano Pacífico, enChiley, al atardecer, enArgentinayUruguay.']
PARAGRAPH: <p id="mwGw">Un <a href="./Eclipse_solar" id="mwHA" rel="mw:WikiLink" title="Eclipse solar">eclipse solar</a> ocurre cua